In [35]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import List, Tuple, Union, Dict

# These are typehints, they mostly make the code readable and testable
t_points = np.array
t_camera_parameters = np.array
t_descriptors = np.array
t_homography = np.array
t_view = np.array
t_img = np.array
t_images = Dict[str, t_img]
t_homographies = Dict[Tuple[str, str], t_homography]  # The keys are the keys of src and destination images

np.set_printoptions(edgeitems=30, linewidth=180,
                    formatter=dict(float=lambda x: "%8.05f" % x))

# Functions

In [144]:
def fast_filter_and_align_descriptors(src: Tuple[t_points, t_descriptors], dst: Tuple[t_points, t_descriptors],
                                      similarity_threshold=0.8) -> Tuple[t_points, t_points]:
    """
    Aligns pairs of keypoints from two images.

    Aligns keypoints from two images based on descriptor similarity using Flann based matcher from cv2
    If K points have been detected in image1 and J points have been detected in image2, the result will be to sets of N
    points representing points with similar descriptors; where N <= J and K <=points.
    Args:
        src: A tuple of two numpy arrays with the first array having dimensions [N x 2] and the second one [N x M]. M
            representing the dimensionality of the point features. In the case of ORB features, M is 32.
        dst: A tuple of two numpy arrays with the first array having dimensions [J x 2] and the second one [J x M]. M
            representing the dimensionality of the point features. In the case of ORB features, M is 32.
        similarity_threshold: The ratio the distance of most similar descriptor to the distance of the second
            most similar.

    Returns:
        A tuple of numpy arrays both sized [N x 2] representing the similar point locations.
    """
    epsilon = .000000001 # for division by zero
    (points1, descriptors1), (points2, descriptors2) = src, dst

    FLANN_INDEX_LSH = 6
    flann_params = dict(algorithm=FLANN_INDEX_LSH, table_number=12, key_size=20, multi_probe_level=3)
    matcher = cv2.FlannBasedMatcher(flann_params, {})
    knnmatches = matcher.knnMatch(descriptors1, descriptors2, 2)

    first_to_second_ratios = np.zeros(len(knnmatches))
    indexes1 = np.zeros(len(knnmatches), dtype=np.int32)
    indexes2 = np.zeros(len(knnmatches), dtype=np.int32)
    for n, (first, second) in enumerate(knnmatches):
        first_to_second_ratios[n] = (first.distance / (second.distance + epsilon))
        indexes1[n] = first.queryIdx
        indexes2[n] = first.trainIdx

    keep_idx = first_to_second_ratios <= similarity_threshold
    filtered_indexes1, filtered_indexes2 = indexes1[keep_idx], indexes2[keep_idx]
    return points1[filtered_indexes1, :], points2[filtered_indexes2, :]

def inliers_epipolar_constraint(p1: t_points, p2: t_points, F:t_homography, distance_threshold: float) -> np.array:
    """Returns indexes of points that satisfy the epipolar constraint

    Args:
        p1: A numpy array of shape [N, 2] representing a single point from image 1
        p2: A numpy array of shape [N, 2] representing a single point from image 2
        F: A numpy array of shape [3, 3] representing the fundamental matrix
        distance_threshold: a float representing the norm of the difference between to points so that they will be
            considered the same (near enough).

    Returns:
        An array of indexes of the points satisfy epipolar constraint.
    """
    ## TODO 2.1 Distance from the Epipolar Line
    # Step 1 - Transform p1 to other image (4 lines)
    # First Convert points1 and points2 from 2D to (2+1)D homogenous coordinates - [x, y] --> [x, y, 1]
    p1_h, p2_h = np.hstack([p1, np.ones((len(p1),1))]), np.hstack([p2, np.ones((len(p2),1))])
    p_proj = np.dot(F, p1_h.T).T

    # Step 2 - normalize the projected points (2 lines)
    p_proj = p_proj/p_proj[:,-1].reshape(-1,1)

    # Step 3 - compute the distance (1 line)
    idx = np.array([k for i,j,k in zip(p2_h,p_proj,range(len(p2_h))) if np.abs(np.dot(i.T,j)) < distance_threshold])
    # Step 4 - check if the distance is smaller than t and return (2 lines)
    return idx


In [171]:
def compute_fundamental_matrix(points1: t_points, points2: t_points) -> np.array:
    """Computes the fundamental matrix given pairs of corresponding points in two images.

    Args:
        points1: A numpy array of size [N x 2] containing x and y coordinates of the source points.
        points2: A numpy array of size [N x 2] containing x and y coordinates of the destination points.

    Returns:
        A [3 x 3] numpy array containing normalised fundamental matrix.
    """
    assert (len(points1) == 8), "Length of points1 should be 8!"
    assert (len(points2) == 8), "Length of points2 should be 8!"

    fundamental_matrix = np.zeros((8, 9)).astype('int')

    # TODO 2.2 Construct the 8x9 matrix A.
    A = np.zeros((8,9)).astype(np.int0)
    for i,p1,p2 in zip(range(len(points1)), points1, points2):
        px, py = p1
        qx, qy = p2
        A[i,:] = [px*qx, px*qy, px, py*qx, py*qy, py, qx, qy, 1]
        
    # TODO 2.3 Solve Af = 0 and extract F using SVD
    _,_,V = np.linalg.svd(A, full_matrices=True)
    V = V.T
    f = V[:,-1]
    F = f.reshape(3,3)
    F = F*(1/f[-1])


    # TODO 2.4 Enforce Rank(F) = 2
    # - Compute the SVD of F
    # - Set sigma3 to 0  (Hint: The singular values are stored as a vector in svd.w)
    # - Recompute F with the updated sigma
    # - Normalize F and store in fundamental matrix
    uf,sf,vf = np.linalg.svd(F, full_matrices=True)
    sf[2:] = 0
    sf = np.diag(sf)
    F_rank2 = np.matmul(np.matmul(uf,sf),vf)
    F = F_rank2.reshape(3,3)
    F = F*(1/F[2,2])

    return F.T

In [155]:
def compute_F_ransac(points1: t_points, points2: t_points, distance_threshold: float = 4.0, steps: int = 1000,
                     n_points: int = 8) -> Tuple[np.array, np.array]:
    """Finds the best homography matrix with the RANSAC algorithm.

    Args:
        points1: A numpy array of size [N x 2] containing x and y coordinates of the source points.
        points2: A numpy array of size [N x 2] containing x and y coordinates of the destination points.
        distance_threshold: a float representing the norm of the difference between to points so that they will be
            considered the same (near enough)
        steps: An integer value representing steps for ransac optimization
        n_points: An integer value representing how many points ransac algorithm to implement

    Returns:
        A tuple with the best homography matrix found and its corresponding inlier indexes.
    """
    best_count = 0
    best_homography = np.eye(3)
    best_inlier_indexes = np.array([])

    for n in tqdm(range(steps)):
        if n == steps - 1:
            print(f"Step: {n:4}  {best_count} RANSAC points match!")

        randomidx = np.random.permutation(points1.shape[0])[:n_points]
        rnd_points1, rnd_points2 = points1[randomidx, :], points2[randomidx, :]

        homography = compute_fundamental_matrix(rnd_points1, rnd_points2)
        inliers_indexes = inliers_epipolar_constraint(points1, points2, homography, distance_threshold)

        if inliers_indexes.shape[0] > best_count:
            best_count = inliers_indexes.shape[0]
            best_homography = homography
            best_inlier_indexes = inliers_indexes
            
    return best_homography, np.array(best_inlier_indexes)


In [168]:
def triangulate(P1: t_view, P2: t_view, p1: t_points, p2: t_points) -> np.array:
    """Projects a point from it's location in the two images to "real-world" 3D coordinates.

    Args:
        P1: A [3 x 4] numpy array representing the camera matrix for View 1 (camera 1 is at 0, 0, 0)
        P2: A [3 x 4] numpy array representing the camera matrix for View 2
        p1: A numpy array of shape [2] representing a single point from image 1
        p2: A numpy array of shape [2] representing a single point from image 2
    Returns:
        resulting_point: A numpy array of shape [3] representing the point in 3D space
    """
    epsilon = .00000001 # avoiding division by zero
    # TODO 3 Triangulation
    # Step 1 - Construct the matrix A (~5 lines)
    # Hint - homogenous solution - Zisserman Book page 312
    px,py = p1
    qx,qy = p2
    A = np.zeros((4,4))
    A[0] = px*P1[2] - P1[0]
    A[1] = py*P1[2] - P1[1]
    A[2] = qx*P2[2] - P2[0]
    A[3] = qy*P2[2] - P2[1]


    # Step 2 - Extract the solution and project it back to real 3D (from homogenous space) (~4 lines)
    _,_,V = np.linalg.svd(A, full_matrices=True)
    V = V.T
    x = V[:-1,-1]/(V[-1,-1]+epsilon)

    return x
#    raise NotImplemented

def triangulate_all_points(View1: t_view, View2: t_view, K: t_view, points1: t_points, points2: t_points) \
        -> t_points:
    """Creates a 3D pointcloud out of corresponding points in two images.

    The pointcloud is also filtered from outliers (points estimated to occur behind the cameras).

    Args:
        View1: A numpy array of shape [3 x 4] representing View matrix 1 for Camera
        View2: A numpy array of shape [3 x 4] representing View matrix 2 for Camera
        K: A numpy array of shape [3 x 3] representing the intrinsic camera parameters
        points1: A numpy array of size [N x 2] containing x and y coordinates of the source points.
        points2: A numpy array of size [N x 2] containing x and y coordinates of the destination points.

    Returns:
        wps: A numpy array of shape [N x 3] representing the point cloud in 3D space
    """
    wps = []
    P1 = np.dot(K, View1)
    P2 = np.dot(K, View2)
    for i in range(len(points1)):
        wp = triangulate(P1, P2, points1[i], points2[i])
        # Check if this points is in front of both cameras
        ptest = [wp[0], wp[1], wp[2], 1]
        p1 = np.matmul(P1, ptest)
        p2 = np.matmul(P2, ptest)
        if (p1[2] > 0) and (p2[2] > 0):
            wps.append(wp)
    wps = np.array(wps)
    return wps


In [175]:
def compute_essential_matrix(fundamental_matrix: t_homography, camera_parameters: t_camera_parameters) -> t_homography:
    """Computes the essential matrix given the fundamental matrix and the intrinsic camera parameters

    Args:
        fundamental_matrix: A [3 x 3] numpy array representing the fundamental matrix
        camera_parameters: A [3 x 3] numpy array representing the instrinsic camera parameters

    Returns:
        essential_matrix: A [3 x 3] numpy array representing the essential matrix
    """
    # TODO 4.1: Calculate essential matrix (2 lines)
    return camera_parameters.T @ fundamental_matrix @ camera_parameters

def decompose(E: t_homography) -> Tuple[np.array, np.array, np.array, np.array]:
    """Decomposes an essential matrix into the two rotation s and 2 translations that constitute it.

    Args:
        E: A [3 x 3] numpy array representing the essential matrix.

    Returns:
        A tuple of R1, R2, t1, t2. R1, R2 are two rotation matrices of shape [3 x 3] and t1, t2 are two translation
            matrices of shape [3]
    """
    # TODO 4.2 Calculating Rotation and translation matrices

    # Step 1 - Compute the SVD E (~1 line)
    u,s,v = np.linalg.svd(E, full_matrices=True)

    # Step 2 - Compute W and Possible rotations (~3-6 lines)
    w = np.array([[0,-1,0],[1,0,0],[0,0,1]])
    r1 = u @ w @ v
    r2 = u @ w.T @ v
    # Step 3 - Possible translations and return R1, R2, t1, t2 (~4 lines)
    t1 = u[2]/np.linalg.norm(u[2])
    t2 = -t1
    return r1,r2,t1,t2

def relativeTransformation(E: t_homography, points1: t_points, points2: t_points, K: t_camera_parameters) -> t_view:
    """Constructs the View Camera Matrix for the second camera position.

    Args:
        E: A [3 x 3] numpy array representing the essential matrix
        points1: A numpy array of size [N x 2] containing x and y coordinates of the source points.
        points2: A numpy array of size [N x 2] containing x and y coordinates of the destination points.
        K: A numpy array of shape [3 x 3] representing the intrinsic camera parameters

    Returns:
        V: A numpy array of shape [3 x 4] representing View matrix 2 for Camera
    """

    R1, R2, t1, t2 = decompose(E)
    ## A negative determinant means that R contains a reflection.This is not rigid transformation!
    if np.linalg.det(R1) < 0:
        E = -E
        R1, R2, t1, t2 = decompose(E)

    bestCount = 0

    for dR in range(2):
        if dR == 0:
            cR = R1
        else:
            cR = R2
        for dt in range(2):
            if dt == 0:
                ct = t1
            else:
                ct = t2

            View1 = np.eye(3, 4)
            View2 = np.zeros((3, 4))
            for i in range(3):
                for j in range(3):
                    View2[i, j] = cR[i, j]
            for i in range(3):
                View2[i, 3] = ct[i]

            count = len(triangulate_all_points(View1, View2, K, points1, points2))
            if (count > bestCount):
                V = View2

    return V


# Testing

In [172]:
def testFundamentalMat():
    points1 = [(1, 1), (3, 7), (2, -5), (10, 11), (11, 2), (-3, 14), (236, -514), (-5, 1)]
    points2 = [(25, 156), (51, -83), (-144, 5), (345, 15),
                                    (215, -156), (151, 83), (1544, 15), (451, -55)]

    F = compute_fundamental_matrix(points1, points2)

    print ("Testing Fundamental Matrix...")
    print ("Your result:" + str(F))

    Href = np.array([[0.001260822171230067,  0.0001614643951166923, -0.001447955678643285],
                 [-0.002080014358205309, -0.002981504896782918, 0.004626528742122177],
                 [-0.8665185546662642,   -0.1168790312603214,   1]])

    print ("Reference: " + str(Href))

    error = Href - F
    e = np.linalg.norm(error)
    print ("Error: " + str(e))

    if (e < 1e-10):
        print ("Test: SUCCESS!")
    else:
        print ("Test: FAIL!")
    print ("============================")

testFundamentalMat()

Testing Fundamental Matrix...
Your result:[[ 0.00126  0.00016 -0.00145]
 [-0.00208 -0.00298  0.00463]
 [-0.86652 -0.11688  1.00000]]
Reference: [[ 0.00126  0.00016 -0.00145]
 [-0.00208 -0.00298  0.00463]
 [-0.86652 -0.11688  1.00000]]
Error: 9.628258568867825e-14
Test: SUCCESS!


In [149]:
points1 = [(1, 1), (3, 7), (2, -5), (10, 11), (11, 2), (-3, 14), (236, -514), (-5, 1)]
points2 = [(25, 156), (51, -83), (-144, 5), (345, 15),
                                    (215, -156), (151, 83), (1544, 15), (451, -55)]

F = np.array([[0.001260822171230067,  0.0001614643951166923, -0.001447955678643285],
                 [-0.002080014358205309, -0.002981504896782918, 0.004626528742122177],
                 [-0.8665185546662642,   -0.1168790312603214,   1]])

inliers_epipolar_constraint(points1,points2,F,0.5)

array([1, 4, 7])

In [169]:
def testTriangulate():
    P1 = np.array([[1, 2, 3, 6], [4, 5, 6, 37], [7, 8, 9, 15]]).astype('float')
    P2 = np.array([[51, 12, 53, 73], [74, 15, -6, -166], [714, -8, 95, 16]]).astype('float')

    F = triangulate(P1, P2, (14.0, 267.0), (626.0, 67.0))
    print("Testing Triangulation...")
    print("Your result: " + str(F))

    wpref = [0.782409, 3.89115, -5.72358]
    print("Reference: " + str(wpref))

    error = wpref - F
    e = cv2.norm(error)
    print("Error: " + str(e))

    if (e < 1e-5):
        print("Test: SUCCESS!")
    else:
        print("Test: FAIL")
    print("================================")
testTriangulate()

Testing Triangulation...
Your result: [ 0.78241  3.89115 -5.72359]
Reference: [0.782409, 3.89115, -5.72358]
Error: 5.369449188447331e-06
Test: SUCCESS!


In [176]:
def testDecompose():
    E = np.array([[3.193843825323984,  1.701122615578195,  -6.27143074201245],
                  [-41.95294882825382, 127.76771801644763, 141.5226870527082],
                  [-8.074440557013252, -134.9928434422784, 1]])
    R1, R2, t1, t2 = decompose(E)

    print("Testing decompose...")

    t_ref = np.array([-0.9973654079783344, -0.04579551552933408, -0.05625845470338976])

    if np.linalg.norm(t1 - t_ref) < 1e-5:
        if np.linalg.norm(t2 + t_ref) < 1e-5:
            print("Test (Translation): SUCCESS")
        else:
            print("Test (Translation): FAIL!")
    elif np.linalg.norm(t1 + t_ref) < 1e-5:
        if np.linalg.norm(t2 - t_ref) < 1e-5:
            print("Test (Translation): SUCCESS")
        else:
            print("Test (Translation): FAIL!")

    R1_ref = np.array([[0.9474295893819155,  -0.1193419720472381, 0.2968748336782551],
                       [0.2288481582901039, 0.9012012887804273, -0.3680553729369057],
                       [-0.2236195286884464, 0.4166458097813701, 0.8811359574894123]])
    R2_ref = np.array([[0.9332689072231527,  0.01099472166665292, 0.3590101153254257],
                       [-0.1424930907107563, -0.9061762138657301, 0.3981713055001164],
                       [0.329704209724715,   -0.4227573601008549, -0.8441394129942975]])

    if np.linalg.norm(R1-R1_ref) < 1e-5:
        if np.linalg.norm(R2-R2_ref) < 1e-5:
            print("Test (Rotation): SUCCESS!")
        else:
            print("Test (Rotation): FAIL!")
    elif np.linalg.norm(R1 - R2_ref) < 1e-5:
        if np.linalg.norm(R2 - R1_ref) < 1e-5:
            print("Test (Rotation): SUCCESS!")
        else:
            print("Test (Rotation): FAIL!")
    print ('='*25)

testDecompose()

Testing decompose...
Test (Rotation): SUCCESS!
